In [1]:
import os
import sys
from pathlib import Path
from copy import deepcopy

sys.path.append(str((Path(os.path.abspath("")) / "..").resolve()))
from sudoku_variants import Sudoku, SudokuAI, SudokuConst
from sudoku_variants.rule import Orthogonal, SubBoard
from sudoku_variants.helper.profile import profile

### AI

In [2]:
board = [[0 for _ in range(SudokuConst.NUM_COL)] for _ in range(SudokuConst.NUM_ROW)]
sudoku = Sudoku(board, [Orthogonal(), SubBoard()])


@profile(1000)
def run():
    SudokuAI.solve(sudoku)


run()

Time: 3.0430s, 1000x | 0.0030s/time


In [3]:
@profile(1000)
def run():
    SudokuAI.generate([Orthogonal(), SubBoard()], max_erased=35)


run()

Time: 3.0086s, 1000x | 0.0030s/time


In [4]:
@profile(1000)
def run():
    SudokuAI.generate([Orthogonal(), SubBoard()], max_erased=40)


run()

Time: 2.9604s, 1000x | 0.0030s/time


In [5]:
@profile(1000)
def run():
    SudokuAI.generate([Orthogonal(), SubBoard()], max_erased=45)


run()

Time: 2.9748s, 1000x | 0.0030s/time


In [6]:
@profile(1000)
def run():
    SudokuAI.generate([Orthogonal(), SubBoard()], max_erased=60)


run()

Time: 2.9979s, 1000x | 0.0030s/time


In [8]:
@profile(1)
def run():
    sudoku = SudokuAI.generate([Orthogonal(), SubBoard()], max_erased=70)
    print(sudoku)


run()

Applied rules: Orthogonal, SubBoard
+-------+-------+-------+
| 8 9 6 | 3 2 1 | 5 4 7 |
| 2 4 7 | 9 6 5 | 8 1 3 |
| 5 3 1 | 4 8 7 | 6 9 2 |
+-------+-------+-------+
| 7 2 5 | 1 9 3 | 4 6 8 |
| 6 1 4 | 2 5 8 | 7 3 9 |
| 3 8 9 | 7 4 6 | 1 2 5 |
+-------+-------+-------+
| 9 7 3 | 8 1 4 | 2 5 6 |
| 4 6 2 | 5 7 9 | 3 8 1 |
| 1 5 8 | 6 3 2 | 9 7 4 |
+-------+-------+-------+
Time: 0.0035s, 1x | 0.0035s/time
